In [8]:
import pandas as pd
import numpy as np

In [2]:
xls_CDNB = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/CDNB_cleaned.xlsx')
df_NB = pd.read_excel(xls_CDNB)
xls_NLQ = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/NLQ_cleaned.xlsx')
df_NLQ = pd.read_excel(xls_NLQ)
df_IT = df_NB.append(df_NLQ,ignore_index=True,sort=False) 

In [3]:
df_IT = df_IT.drop(df_IT[(df_IT.RegisterBuyVolume.isnull())&(df_IT.BuyVolume == 0)].index)
df_IT = df_IT.drop(df_IT[(df_IT.RegisterSellVolume.isnull())&(df_IT.SellVolume == 0)].index)

In [4]:
xls_1220 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2012_2020.xlsx')
xls_11 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2011.xlsx')
xls_21 = pd.ExcelFile('/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Trading_2021.xlsx')

df1_1220 = pd.read_excel(xls_1220, '1')
df2_1220 = pd.read_excel(xls_1220, '2')
df3_1220 = pd.read_excel(xls_1220, '3')
df4_11 = pd.read_excel(xls_11)
df5_21 = pd.read_excel(xls_21)

#Merge all into df_1121
df_1121 = df1_1220.append([df2_1220,df3_1220,df4_11,df5_21],ignore_index=True,sort=False)
df_1121_dup = df_1121.copy()
df_1121_dup.rename(columns={"TradingDate":"RegisterDate"}, inplace=True)

#Calculate daily ret of securities
df_1121_dup['Pi_ret'] = df_1121_dup.groupby('StockCode')['CloseAdj'].pct_change(-1)

In [5]:
#Map 'Exchange' & 'KLCPGD' values using cols
cols =['StockCode','RegisterDate']
df_IT['Exchange'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Exchange'])
df_IT['KLCPLH'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['KLCPLH'])

#Calculate date difference
from datetime import datetime
A = [d.date() for d in df_IT['DateStartExpected']]
B = [d.date() for d in df_IT['RegisterDate']]
df_IT['Diff_days'] = np.busday_count(B, A)

#Recalculate percentage values
df_IT['RegisterVolumeBeforePercent'] = df_IT['RegisterVolumeBefore']/df_IT['KLCPLH']
df_IT['RegisterVolumeAfterPercent'] = df_IT['RegisterVolumeAfter']/df_IT['KLCPLH']
df_IT['VolumeBeforePercent'] = df_IT['VolumeBefore']/df_IT['KLCPLH']
df_IT['VolumeAfterPercent'] = df_IT['VolumeAfter']/df_IT['KLCPLH']

#Cleaning data: drop all percentage values>1
df_IT = df_IT[(df_IT['RegisterVolumeBeforePercent'] < 1) &
               (df_IT['RegisterVolumeAfterPercent'] < 1) &
               (df_IT['VolumeBeforePercent'] < 1) & (df_IT['VolumeAfterPercent']<1)]

In [6]:
#Match Securities Ret

df_IT['Ri_T1_0'] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols)['Pi_ret'])

for x in range(1,12):
    df_IT['Ri_T1_' + str(x)] = df_IT.set_index(cols).index.map(df_1121_dup.set_index(cols).groupby('StockCode')['Pi_ret'].shift(x))

In [7]:
df_IT.to_excel(r'/Users/hongnhungvu/Desktop/preview_12_all.xlsx',index=False)

#Here I export the file to clear all stock prices that does not belong [t,t+n], the file above is [t,t+12) and n is in [0,10]
#I use filter in Excel and clear manually
#Since I don't know how to do it in Python

In [9]:
xls_new = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/preview_12_all.xlsx')
df_new = pd.read_excel(xls_new)

In [10]:
df_new_buy = pd.DataFrame(df_new[(df_new['RegisterSellVolume'].isnull())])
df_new_sell = pd.DataFrame(df_new[(df_new['RegisterBuyVolume'].isnull())])

In [11]:
for i in range(len(df_new_buy)):
    k = df_new_buy['Diff_days'].iloc[i]
    df_new_buy.iloc[i,32+k] = (df_new_buy.iloc[i,32+k]*(-1))*k
df_new_buy

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,Ri_T1_1,Ri_T1_2,Ri_T1_3,Ri_T1_4,Ri_T1_5,Ri_T1_6,Ri_T1_7,Ri_T1_8,Ri_T1_9,Ri_T1_10
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,0.018272,...,0.000000,-0.011765,-0.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,0.032308,...,0.000000,-0.017241,-0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PNC,2012-02-03,2012-02-17,2012-02-08,2012-03-08,2012-02-08,2012-02-15,GD CD noi bo,310000,0.031575,...,0.046512,0.022222,-0.130435,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LSS,2012-02-13,2012-02-17,2012-02-16,2012-02-28,2012-02-16,2012-02-16,GD CD noi bo,23462,0.000469,...,0.013889,-0.018265,-0.027907,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DDM,2012-02-07,2012-02-20,2012-02-10,2012-03-31,2012-02-10,2012-02-16,GD CD noi bo,0,0.000000,...,0.037037,0.035714,0.103448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,TVC,2020-12-11,2020-12-23,2020-12-14,2021-01-08,2020-12-14,2020-12-21,GD cua nguoi lien quan,16104073,0.349739,...,0.016000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11910,PGC,2020-11-19,2020-12-24,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,5066456,0.083966,...,0.022026,0.077586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11911,DTL,2020-12-11,2020-12-24,2020-12-17,2021-01-16,2020-12-23,2020-12-24,GD cua nguoi lien quan,235000,0.003876,...,0.068562,0.068858,0.068814,-0.279452,NaN,NaN,NaN,NaN,NaN,NaN
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,0.000000,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
ret_buy = np.nansum(df_new_buy.iloc[:,32:43],axis=1)
ret_buy

array([-0.15462185, -0.09595586, -0.08442821, ..., -0.00345109,
        0.12403101, -0.05641654])

In [13]:
df_new_buy['ret_buy'] = np.array(ret_buy)
df_new_buy

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,Ri_T1_2,Ri_T1_3,Ri_T1_4,Ri_T1_5,Ri_T1_6,Ri_T1_7,Ri_T1_8,Ri_T1_9,Ri_T1_10,ret_buy
1,DHA,2012-01-05,2012-02-06,2012-01-10,2012-03-09,2012-01-10,2012-02-03,GD CD noi bo,275206,0.018272,...,-0.011765,-0.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.154622
2,REE,2012-01-04,2012-02-13,2012-01-09,2012-03-09,2012-01-09,2012-02-13,GD CD noi bo,7742346,0.032308,...,-0.017241,-0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.095956
7,PNC,2012-02-03,2012-02-17,2012-02-08,2012-03-08,2012-02-08,2012-02-15,GD CD noi bo,310000,0.031575,...,0.022222,-0.130435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.084428
8,LSS,2012-02-13,2012-02-17,2012-02-16,2012-02-28,2012-02-16,2012-02-16,GD CD noi bo,23462,0.000469,...,-0.018265,-0.027907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.036891
9,DDM,2012-02-07,2012-02-20,2012-02-10,2012-03-31,2012-02-10,2012-02-16,GD CD noi bo,0,0.000000,...,0.035714,0.103448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.176200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,TVC,2020-12-11,2020-12-23,2020-12-14,2021-01-08,2020-12-14,2020-12-21,GD cua nguoi lien quan,16104073,0.349739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040590
11910,PGC,2020-11-19,2020-12-24,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,5066456,0.083966,...,0.077586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.113005
11911,DTL,2020-12-11,2020-12-24,2020-12-17,2021-01-16,2020-12-23,2020-12-24,GD cua nguoi lien quan,235000,0.003876,...,0.068858,0.068814,-0.279452,NaN,NaN,NaN,NaN,NaN,NaN,-0.003451
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.124031


In [14]:
df_new_buy['year']=df_new_buy['RegisterDate'].dt.year
df_new_buy['month']=df_new_buy['RegisterDate'].dt.month

In [15]:
import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

w, p = wilcoxon(df_new_buy['ret_buy'], alternative='greater')
w, p, norm.ppf(p), df_new_buy['ret_buy'].median()

(5842201.0, 1.904908759205769e-06, -4.621496505763976, 0.0)

In [16]:
import scipy.integrate 

from scipy import stats
result = scipy.stats.ttest_1samp(df_new_buy['ret_buy'],0, alternative='greater')
result

Ttest_1sampResult(statistic=3.012053478201232, pvalue=0.0013035761358723128)

In [17]:
df_VNI_12 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2012)]

w_VNI_12, p_VNI_12 = wilcoxon(df_VNI_12['ret_buy'], alternative='greater')
w_VNI_12, p_VNI_12, norm.ppf(p_VNI_12), df_VNI_12['ret_buy'].median()

(26837.0, 0.025824506081636448, -1.9460476825671702, 0.0017849011189177214)

In [18]:
df_VNI_13 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2013)]

w_VNI_13, p_VNI_13 = wilcoxon(df_VNI_13['ret_buy'], alternative='greater')
w_VNI_13, p_VNI_13, norm.ppf(p_VNI_13), df_VNI_13['ret_buy'].median()

(17737.0, 0.11469641874088077, -1.2019243546513068, 0.010445891456683178)

In [19]:
df_VNI_14 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2014)]

w_VNI_14, p_VNI_14 = wilcoxon(df_VNI_14['ret_buy'], alternative='greater')
w_VNI_14, p_VNI_14, norm.ppf(p_VNI_14), df_VNI_14['ret_buy'].median()

(17613.0, 0.4181925405652328, -0.20651956799942653, 0.0028432224947252463)

In [20]:
df_VNI_15 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2015)]

w_VNI_15, p_VNI_15 = wilcoxon(df_VNI_15['ret_buy'], alternative='greater')
w_VNI_15, p_VNI_15, norm.ppf(p_VNI_15), df_VNI_15['ret_buy'].median()

(18306.0, 0.16068895507437908, -0.9916302851752318, 0.0041753835425383246)

In [21]:
df_VNI_16 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2016)]

w_VNI_16, p_VNI_16 = wilcoxon(df_VNI_16['ret_buy'], alternative='greater')
w_VNI_16, p_VNI_16, norm.ppf(p_VNI_16), df_VNI_16['ret_buy'].median()

(26045.0, 0.014027677260117274, -2.196511478160663, 0.007328912218576568)

In [22]:
df_VNI_17 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2017)]

w_VNI_17, p_VNI_17 = wilcoxon(df_VNI_17['ret_buy'], alternative='greater')
w_VNI_17, p_VNI_17, norm.ppf(p_VNI_17), df_VNI_17['ret_buy'].median()

(29344.0, 0.42154979710125723, -0.1979303037775547, 8.140008139978071e-06)

In [23]:
df_VNI_18 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2018)]

w_VNI_18, p_VNI_18 = wilcoxon(df_VNI_18['ret_buy'], alternative='greater')
w_VNI_18, p_VNI_18, norm.ppf(p_VNI_18), df_VNI_18['ret_buy'].median()

(58814.0, 0.08910573124053461, -1.3462823911753647, 0.002329023931679556)

In [24]:
df_VNI_19 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2019)]

w_VNI_19, p_VNI_19 = wilcoxon(df_VNI_19['ret_buy'], alternative='greater')
w_VNI_19, p_VNI_19, norm.ppf(p_VNI_19), df_VNI_19['ret_buy'].median()

(65350.0, 0.04071407384845693, -1.7424590687785169, 0.00576512024812725)

In [25]:
df_VNI_20 = df_new_buy[(df_new_buy['Exchange']=='HOSE')&(df_new_buy['year']==2020)]

w_VNI_20, p_VNI_20 = wilcoxon(df_VNI_20['ret_buy'], alternative='greater')
w_VNI_20, p_VNI_20, norm.ppf(p_VNI_20), df_VNI_20['ret_buy'].median()

(60244.0, 0.2032331821970616, -0.8301280943149283, 0.00026172651772138344)

In [26]:
df_HNX_12 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2012)]

w_HNX_12, p_HNX_12 = wilcoxon(df_HNX_12['ret_buy'], alternative='greater')
w_HNX_12, p_HNX_12, norm.ppf(p_HNX_12), df_HNX_12['ret_buy'].median()

(349.0, 0.6224033302371749, 0.31179892703917605, 0.0)

In [27]:
df_HNX_13 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2013)]

w_HNX_13, p_HNX_13 = wilcoxon(df_HNX_13['ret_buy'], alternative='greater')
w_HNX_13, p_HNX_13, norm.ppf(p_HNX_13), df_HNX_13['ret_buy'].median()

(1390.5, 0.7134882645675222, 0.5636042836906145, 0.0)

In [28]:
df_HNX_14 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2014)]

w_HNX_14, p_HNX_14 = wilcoxon(df_HNX_14['ret_buy'], alternative='greater')
w_HNX_14, p_HNX_14, norm.ppf(p_HNX_14), df_HNX_14['ret_buy'].median()

(5901.0, 0.11485938605663437, -1.2010836048698508, 0.0)

In [29]:
df_HNX_15 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2015)]

w_HNX_15, p_HNX_15 = wilcoxon(df_HNX_15['ret_buy'], alternative='greater')
w_HNX_15, p_HNX_15, norm.ppf(p_HNX_15), df_HNX_15['ret_buy'].median()

(9568.0, 4.57626141850131e-05, -3.9120255434806963, 0.0182516706195891)

In [30]:
df_HNX_16 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2016)]

w_HNX_16, p_HNX_16 = wilcoxon(df_HNX_16['ret_buy'], alternative='greater')
w_HNX_16, p_HNX_16, norm.ppf(p_HNX_16), df_HNX_16['ret_buy'].median()

(3963.0, 0.5960182721743073, 0.24305414173728462, 0.0)

In [31]:
df_HNX_17 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2017)]

w_HNX_17, p_HNX_17 = wilcoxon(df_HNX_17['ret_buy'], alternative='greater')
w_HNX_17, p_HNX_17, norm.ppf(p_HNX_17), df_HNX_17['ret_buy'].median()

(2028.0, 0.726904685012225, 0.6034781754422118, 0.0)

In [32]:
df_HNX_18 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2018)]

w_HNX_18, p_HNX_18 = wilcoxon(df_HNX_18['ret_buy'], alternative='greater')
w_HNX_18, p_HNX_18, norm.ppf(p_HNX_18), df_HNX_18['ret_buy'].median()

(1180.0, 0.825715536079878, 0.9373687149428296, 0.0)

In [33]:
df_HNX_19 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2019)]

w_HNX_19, p_HNX_19 = wilcoxon(df_HNX_19['ret_buy'], alternative='greater')
w_HNX_19, p_HNX_19, norm.ppf(p_HNX_19), df_HNX_19['ret_buy'].median()

(2571.0, 0.10089223623413718, -1.2764840191006814, 0.0)

In [34]:
df_HNX_20 = df_new_buy[(df_new_buy['Exchange']=='HNX')&(df_new_buy['year']==2020)]

w_HNX_20, p_HNX_20 = wilcoxon(df_HNX_20['ret_buy'], alternative='greater')
w_HNX_20, p_HNX_20, norm.ppf(p_HNX_20), df_HNX_20['ret_buy'].median()

(2188.0, 0.003221639900414161, -2.7243263780916607, 0.009208133184844815)

In [35]:
df_UPCOM_16 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2016)]

w_UPCOM_16, p_UPCOM_16 = wilcoxon(df_UPCOM_16['ret_buy'], alternative='greater')
w_UPCOM_16, p_UPCOM_16, norm.ppf(p_UPCOM_16), df_UPCOM_16['ret_buy'].median()

(1538.0, 0.864679937836385, 1.1015896169800565, 0.0)

In [36]:
df_UPCOM_17 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2017)]

w_UPCOM_17, p_UPCOM_17 = wilcoxon(df_UPCOM_17['ret_buy'], alternative='greater')
w_UPCOM_17, p_UPCOM_17, norm.ppf(p_UPCOM_17), df_UPCOM_17['ret_buy'].median()

(3890.0, 0.422948144513845, -0.1943570866752046, 0.0)

In [37]:
df_UPCOM_18 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2018)]

w_UPCOM_18, p_UPCOM_18 = wilcoxon(df_UPCOM_18['ret_buy'], alternative='greater')
w_UPCOM_18, p_UPCOM_18, norm.ppf(p_UPCOM_18), df_UPCOM_18['ret_buy'].median()

(3303.0, 0.28306670542348844, -0.5737552855115068, 0.0)

In [38]:
df_UPCOM_19 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2019)]

w_UPCOM_19, p_UPCOM_19 = wilcoxon(df_UPCOM_19['ret_buy'], alternative='greater')
w_UPCOM_19, p_UPCOM_19, norm.ppf(p_UPCOM_19), df_UPCOM_19['ret_buy'].median()

(2871.0, 0.023608580497311873, -1.984347081230126, 0.0)

In [39]:
df_UPCOM_20 = df_new_buy[(df_new_buy['Exchange']=='UPCOM')&(df_new_buy['year']==2020)]

w_UPCOM_20, p_UPCOM_20 = wilcoxon(df_UPCOM_20['ret_buy'], alternative='greater')
w_UPCOM_20, p_UPCOM_20, norm.ppf(p_UPCOM_20), df_UPCOM_20['ret_buy'].median()

(3414.0, 0.10397750856517546, -1.2592085417001995, 0.0)

In [40]:
tstat_VNI_12, pval_VNI_12 = scipy.stats.ttest_1samp(df_VNI_12['ret_buy'],0, alternative='greater')
tstat_VNI_12, pval_VNI_12, df_VNI_12['ret_buy'].mean(), df_VNI_12['ret_buy'].std()

(2.4745996902426426,
 0.006934944383521355,
 0.014386649362089929,
 0.102855346945238)

In [41]:
tstat_VNI_13, pval_VNI_13 = scipy.stats.ttest_1samp(df_VNI_13['ret_buy'],0, alternative='greater')
tstat_VNI_13, pval_VNI_13, df_VNI_13['ret_buy'].mean(), df_VNI_13['ret_buy'].std()

(1.2655003932047484,
 0.10341683347776345,
 0.010917260905580364,
 0.13883574242183866)

In [42]:
tstat_VNI_14, pval_VNI_14 = scipy.stats.ttest_1samp(df_VNI_14['ret_buy'],0, alternative='greater')
tstat_VNI_14, pval_VNI_14, df_VNI_14['ret_buy'].mean(), df_VNI_14['ret_buy'].std()

(0.3228026957743863,
 0.37354960957146294,
 0.002269659942668591,
 0.11467386708387317)

In [43]:
tstat_VNI_15, pval_VNI_15 = scipy.stats.ttest_1samp(df_VNI_15['ret_buy'],0, alternative='greater')
tstat_VNI_15, pval_VNI_15, df_VNI_15['ret_buy'].mean(), df_VNI_15['ret_buy'].std()

(0.5934281225107048,
 0.27670233578647807,
 0.0033309715136108876,
 0.09120208320034608)

In [44]:
tstat_VNI_16, pval_VNI_16 = scipy.stats.ttest_1samp(df_VNI_16['ret_buy'],0, alternative='greater')
tstat_VNI_16, pval_VNI_16, df_VNI_16['ret_buy'].mean(), df_VNI_16['ret_buy'].std()

(2.3250578371094854,
 0.010366820766479627,
 0.014698117465286892,
 0.10985793331827172)

In [45]:
tstat_VNI_17, pval_VNI_17 = scipy.stats.ttest_1samp(df_VNI_17['ret_buy'],0, alternative='greater')
tstat_VNI_17, pval_VNI_17, df_VNI_17['ret_buy'].mean(), df_VNI_17['ret_buy'].std()

(-0.24228610703462708,
 0.5956481481794402,
 -0.001377920656613941,
 0.10532773834534405)

In [46]:
tstat_VNI_18, pval_VNI_18 = scipy.stats.ttest_1samp(df_VNI_18['ret_buy'],0, alternative='greater')
tstat_VNI_18, pval_VNI_18, df_VNI_18['ret_buy'].mean(), df_VNI_18['ret_buy'].std()

(0.5241478015090361,
 0.3002098586953421,
 0.002327605729257521,
 0.09698729312110763)

In [47]:
tstat_VNI_19, pval_VNI_19 = scipy.stats.ttest_1samp(df_VNI_19['ret_buy'],0, alternative='greater')
tstat_VNI_19, pval_VNI_19, df_VNI_19['ret_buy'].mean(), df_VNI_19['ret_buy'].std()

(0.7189217488097018,
 0.23626521734088607,
 0.003525181865284961,
 0.10887379588407989)

In [48]:
tstat_VNI_20, pval_VNI_20 = scipy.stats.ttest_1samp(df_VNI_20['ret_buy'],0, alternative='greater')
tstat_VNI_20, pval_VNI_20, df_VNI_20['ret_buy'].mean(), df_VNI_20['ret_buy'].std()

(0.36491959723417067,
 0.35766541173380023,
 0.0020183204538784883,
 0.12180463195841583)

In [49]:
tstat_HNX_13, pval_HNX_13 = scipy.stats.ttest_1samp(df_HNX_13['ret_buy'],0, alternative='greater')
tstat_HNX_13, pval_HNX_13, df_HNX_13['ret_buy'].mean(), df_HNX_13['ret_buy'].std()

(-0.8593807131893851,
 0.8037672377930754,
 -0.010822313993674351,
 0.11880358082811239)

In [50]:
tstat_HNX_14, pval_HNX_14 = scipy.stats.ttest_1samp(df_HNX_14['ret_buy'],0, alternative='greater')
tstat_HNX_14, pval_HNX_14, df_HNX_14['ret_buy'].mean(), df_HNX_14['ret_buy'].std()

(1.3917729311974874,
 0.08298619343841866,
 0.016091384247929035,
 0.14486879447308848)

In [51]:
tstat_HNX_15, pval_HNX_15 = scipy.stats.ttest_1samp(df_HNX_15['ret_buy'],0, alternative='greater')
tstat_HNX_15, pval_HNX_15, df_HNX_15['ret_buy'].mean(), df_HNX_15['ret_buy'].std()

(2.4087830569874624,
 0.008472924310702836,
 0.02178948027258907,
 0.12599405360075316)

In [52]:
tstat_HNX_16, pval_HNX_16 = scipy.stats.ttest_1samp(df_HNX_16['ret_buy'],0, alternative='greater')
tstat_HNX_16, pval_HNX_16, df_HNX_16['ret_buy'].mean(), df_HNX_16['ret_buy'].std()

(0.6697779962952117,
 0.25203595015078284,
 0.007567511551678311,
 0.13605240330330806)

In [53]:
tstat_HNX_17, pval_HNX_17 = scipy.stats.ttest_1samp(df_HNX_17['ret_buy'],0, alternative='greater')
tstat_HNX_17, pval_HNX_17, df_HNX_17['ret_buy'].mean(), df_HNX_17['ret_buy'].std()

(-0.6792636292117439,
 0.7508437893306592,
 -0.007788526557872922,
 0.12455407695865772)

In [54]:
tstat_HNX_19, pval_HNX_19 = scipy.stats.ttest_1samp(df_HNX_19['ret_buy'],0, alternative='greater')
tstat_HNX_19, pval_HNX_19, df_HNX_19['ret_buy'].mean(), df_HNX_19['ret_buy'].std()

(1.251257639002068,
 0.10668996606231483,
 0.01650380835132182,
 0.1420582382827457)

In [55]:
tstat_UPCOM_16, pval_UPCOM_16 = scipy.stats.ttest_1samp(df_UPCOM_16['ret_buy'],0, alternative='greater')
tstat_UPCOM_16, pval_UPCOM_16, df_UPCOM_16['ret_buy'].mean(), df_UPCOM_16['ret_buy'].std()

(-1.4995308714431108,
 0.9318508343580943,
 -0.0319283773811882,
 0.2371003960390449)

In [56]:
tstat_UPCOM_17, pval_UPCOM_17 = scipy.stats.ttest_1samp(df_UPCOM_17['ret_buy'],0, alternative='greater')
tstat_UPCOM_17, pval_UPCOM_17, df_UPCOM_17['ret_buy'].mean(), df_UPCOM_17['ret_buy'].std()

(0.4979784080659804,
 0.3095587072373583,
 0.0049850265544394675,
 0.13355709517282602)

In [57]:
tstat_UPCOM_18, pval_UPCOM_18 = scipy.stats.ttest_1samp(df_UPCOM_18['ret_buy'],0, alternative='greater')
tstat_UPCOM_18, pval_UPCOM_18, df_UPCOM_18['ret_buy'].mean(), df_UPCOM_18['ret_buy'].std()

(-0.9257086013394045,
 0.8221047297079527,
 -0.02034177906514355,
 0.3012961336353891)

In [58]:
tstat_UPCOM_19, pval_UPCOM_19 = scipy.stats.ttest_1samp(df_UPCOM_19['ret_buy'],0, alternative='greater')
tstat_UPCOM_19, pval_UPCOM_19, df_UPCOM_19['ret_buy'].mean(), df_UPCOM_19['ret_buy'].std()

(1.3118463975105197,
 0.09560762588293512,
 0.019867634497778435,
 0.20543388830242384)

In [59]:
tstat_UPCOM_20, pval_UPCOM_20 = scipy.stats.ttest_1samp(df_UPCOM_20['ret_buy'],0, alternative='greater')
tstat_UPCOM_20, pval_UPCOM_20, df_UPCOM_20['ret_buy'].mean(), df_UPCOM_20['ret_buy'].std()

(1.6296598076416218,
 0.05232567970592972,
 0.021956070450414145,
 0.1970899810064331)